In [64]:
import base64
import requests
import numpy as np
import matplotlib.pyplot as plt
import glob
import torch
import os
import re
import pickle 
import time
from collections import Counter
from natsort import natsorted

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff


import sys
sys.path.insert(0, '..')
from utils_bnr import set_seed

class Args:
    seed = 0
    batch_size = 1
    num_workers = 4
    image_size = 128
    image_channels = 3

    num_slots = 4
    
    #-----------------------------------------------------------------#
    #!!!!!!!!!!!!!!!!Change this as necessary!!!!!!!!!!!!!!!!!!!!!!!!!!
    #-----------------------------------------------------------------#
#     base_path = '../logs/clevr_easy_500_epochs/sysbind_orig_seed0'
#     base_path = '../logs/clevr_easy_500_epochs/sysbind_orig_seed1'
#     base_path = '../logs/clevr_easy_500_epochs/sysbind_orig_seed2'
#     rand_imgs_path = '../logs/clevr_easy_500_epochs/random_imgs'
#     num_blocks = 8
    #-------------------------------#
#     base_path = '../logs/clevr4_600_epochs/clevr4_sysbind_orig_seed0'
    base_path = '../logs/clevr4_600_epochs/clevr4_sysbind_orig_seed1'
#     base_path = '../logs/clevr4_600_epochs/clevr4_sysbind_orig_seed2'
    rand_imgs_path = '../logs/clevr4_600_epochs/random_imgs'
    num_blocks = 16
    #-----------------------------------------------------------------#

args=Args()
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(args.seed)

# set seed
torch.manual_seed(args.seed)

revise_dir = f'{args.base_path}/revision'
if not os.path.exists(revise_dir):
    os.makedirs(revise_dir)
args.save_path = revise_dir

In [65]:
# OpenAI API Key
api_key = "your-key"

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

In [66]:
def extract_answer_from_response_json(response, split_str='Final Answer: ', verbose=0):
    """
    response: response from the VLM model.
    split_str: string that specifies where to cut the VLM response in order to extract the final task answer.
    """
    try:
        answer = response.json()['choices'][0]['message']['content']
    except:
        print(f'{response.json()} \n Error')
    
    if verbose:
        print(answer)
    
    answer_y_n = answer.split(split_str)[-1]
    if 'No' not in answer_y_n and 'Yes' not in answer_y_n:
        print(answer)        
        raise Exception("This response isn't binary") 
    else:
        if 'No' in answer_y_n:
            return 0
        elif 'Yes' in answer_y_n:
            return 1
     
    
def extract_properties_from_response(response):
     return '{'+response.json()['choices'][0]['message']['content'].split('{\n')[-1].split('\n}')[0]+'}'
    
    
# def extract_obj_list_from_response(response):
#     # get object dict as str from response
#     obj_dict_str = response.json()['choices'][0]['message']['content'].split('{')[-1].split('}')[0]
#     # extract only property description lists
#     obj_list_str = re.split(r'Object\d+: ', obj_dict_str)[1:]
#     obj_list_str = [re.split(r'], ', s)[0]+']' for s in obj_list_str] 
#     obj_list_str = [s.split('[')[1].split(']')[0].split(', ') for s in obj_list_str]
#     return obj_list_str
    
    
def find_majority(responses, split_str='Final Answer: ' , verbose=0):
    """
    responses: list of response json from VLM model. This list should be the list of multiply querying the model 
    with the same sample and prompt.
    
    returns: the majority vote from these responses.
    """
    
    votes = [extract_answer_from_response_json(response, split_str, verbose) for response in responses]
    
    vote_count = Counter(votes)
    top_two = vote_count.most_common(2)
    if len(top_two)>1 and top_two[0][1] == top_two[1][1]:
        # It is a tie
        return 0
    return top_two[0][0]


# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
    
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)

## 1. Query VLM for property lists - N times

In [67]:
# knowledge_prompt = """You are provided six images. An image contains subimages. Each subimage depicts one object. Each object represents a reflective geometric solid that is placed in a neutral gray background scene with a light source. Furthermore, each object has multiple properties, e.g., color. Each property can be subdivided into several sub-properties, e.g., brown is a sub-property of the property color. 

# Please provide a list of obect properties and subproperties that are depicted in all images. Ignore the background and the object's luminance and reflectivity. Use the following answer template:

# {
# property: [sub-property, sub-property, ...]
# property: [sub-property, sub-property, ...]
# ...
# }
# """
knowledge_prompt = """You are provided six images. An image contains subimages. Each subimage depicts one object. Each object represents a reflective geometric solid that is placed in a neutral gray background scene with a light source. Furthermore, each object has multiple properties, e.g., color, shape, size, material. Each property can be subdivided into several sub-properties, e.g., brown is a sub-property of the property color. 

Please provide a list of obect properties and subproperties that are depicted in all images. Ignore the background and the object's luminance and reflectivity. Use the following answer template:

{
property: [sub-property, sub-property, ...]
property: [sub-property, sub-property, ...]
...
}
"""

In [68]:
N_QUERY_SAMPLES = 10

RAND_IMG_PTH_0 = f"{args.rand_imgs_path}/random_imgs_0.png"
BASE64_RAND_IMG_0 = encode_image(RAND_IMG_PTH_0)
RAND_IMG_PTH_1 = f"{args.rand_imgs_path}/random_imgs_1.png"
BASE64_RAND_IMG_1 = encode_image(RAND_IMG_PTH_1)
RAND_IMG_PTH_2 = f"{args.rand_imgs_path}/random_imgs_2.png"
BASE64_RAND_IMG_2 = encode_image(RAND_IMG_PTH_2)
RAND_IMG_PTH_3 = f"{args.rand_imgs_path}/random_imgs_3.png"
BASE64_RAND_IMG_3 = encode_image(RAND_IMG_PTH_3)
RAND_IMG_PTH_4 = f"{args.rand_imgs_path}/random_imgs_4.png"
BASE64_RAND_IMG_4 = encode_image(RAND_IMG_PTH_4)
RAND_IMG_PTH_5 = f"{args.rand_imgs_path}/random_imgs_5.png"
BASE64_RAND_IMG_5 = encode_image(RAND_IMG_PTH_5)

knowledge_responses = []
knowledge_responses_lists = [] # only contains the lists as string
# first ask for the property list:
for n_sample in range(N_QUERY_SAMPLES):
    payload = {
      "model": "gpt-4-vision-preview",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": f"{knowledge_prompt}",
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{BASE64_RAND_IMG_0}"
              }
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{BASE64_RAND_IMG_1}"
              }
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{BASE64_RAND_IMG_2}"
              }
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{BASE64_RAND_IMG_3}"
              }
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{BASE64_RAND_IMG_4}"
              }
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{BASE64_RAND_IMG_5}"
              }
            },
          ],
        }
      ],
      "seed":0,
      "max_tokens": 400,
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    print(response.json())
    response_str = extract_properties_from_response(response)
    print(response_str)

    knowledge_responses.append(response)
    knowledge_responses_lists.append(response_str)
    
    time.sleep(40)


{'id': 'chatcmpl-9MYEluoq2PFGe5FPldmWGuoo1er7s', 'object': 'chat.completion', 'created': 1715161851, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\nshape: [sphere, cube, cylinder, cone]\ncolor: [red, green, blue, yellow, purple, teal, brown, gray]\nsize: [small, medium, large]\nmaterial: [matte, shiny, transparent]\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1683, 'completion_tokens': 51, 'total_tokens': 1734}, 'system_fingerprint': None}
{shape: [sphere, cube, cylinder, cone]
color: [red, green, blue, yellow, purple, teal, brown, gray]
size: [small, medium, large]
material: [matte, shiny, transparent]}
{'id': 'chatcmpl-9MYFUeMBdrTyThna2IMztQbNwrL49', 'object': 'chat.completion', 'created': 1715161896, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\nshape: [sphere, cube, cylinder, cone]\ncolor: [red, green, blue, yellow, purple

In [69]:
with open(f'{args.save_path}/responses_knowledge_properties.pkl', 'wb') as f:
    pickle.dump(knowledge_responses, f)

## 2. Query VLM for occurence of sub-property per object in image

In [92]:
# load property list responses
with open(f'{args.save_path}/responses_knowledge_properties.pkl', 'rb') as f:
    knowledge_responses = pickle.load(f)

In [93]:
N_QUERY_SAMPLES = 4

responses_description = {}
# for n_sample in range(N_QUERY_SAMPLES):
for n_sample in [3]:
    print(f'\n-------------------------------\nQuery: {n_sample}\n\n')

    properties_str = extract_properties_from_response(knowledge_responses[n_sample])
    
    description_prompt = f"You are provided an image. The image contains at most 25 subimages. Each subimage depicts one object. Each object represents a reflective geometric solid that is placed in a neutral gray background scene with a light source. Furthermore, each object has multiple properties, e.g., color. Each property can be subdivided into several sub-properties, e.g., green is a sub-property of the property color. The possible properties and sub-properties are the following: \n\n{properties_str}\n\nFocusing only on these properties, please perform the following tasks.\n\nFirst, for every object in the image please list the sub-properties from the given lists that the object depicts. Only name the sub-properties that are given. Please use the following format:\n\n{{Object1: [sub-property, ...], Object2: [sub-property, ...], ...\n}}"

    responses_block = {}
    # Gather the responses per concept deletion query
    for block_id in range(args.num_blocks):

        image_path = f"{args.base_path}/clustered_exemplars/block{block_id}*.png"
        exemplar_paths = glob.glob(image_path)
        exemplar_paths = natsorted(exemplar_paths)
        n_clusters = len(exemplar_paths)

        responses_concept = {}
        
        if n_clusters > 1:

            for concept_id, img_pth in enumerate(exemplar_paths):

                print(f'{block_id}: {concept_id}')

                # Getting the base64 string
                base64_concept_image = encode_image(img_pth)

                payload = {
                  "model": "gpt-4-vision-preview",
                  "messages": [
                    {
                      "role": "user",
                      "content": [
                        {
                          "type": "text",
                          "text": f"{description_prompt}",
                        },
                        {
                          "type": "image_url",
                          "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_concept_image}"
                          }
                        },
                      ],
                    }
                  ],
                  "max_tokens": 400
                }

                response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
                responses_concept[concept_id] = response
                print(response.json())

                time.sleep(30)

        responses_block[block_id] = responses_concept

    responses_description[n_sample] = responses_block


-------------------------------
Query: 3


0: 0
{'id': 'chatcmpl-9NEwdblMtTcXwg536tZfSOEiZIjoY', 'object': 'chat.completion', 'created': 1715326019, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\nObject1: [cone, blue, medium, matte],\nObject2: [cube, teal, medium, metallic],\nObject3: [cube, yellow, medium, matte],\nObject4: [cube, red, medium, matte],\nObject5: [cylinder, teal, medium, metallic],\nObject6: [cylinder, gray, medium, matte],\nObject7: [cube, brown, medium, metallic],\nObject8: [cube, yellow, medium, metallic],\nObject9: [cylinder, brown, medium, metallic],\nObject10: [cone, purple, medium, matte]\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 646, 'completion_tokens': 125, 'total_tokens': 771}, 'system_fingerprint': None}
0: 1
{'id': 'chatcmpl-9NExWeLqvvYRv0nsAKaCblRtqoaTL', 'object': 'chat.completion', 'created': 1715326074, 'model': 'gpt-4-1106-vision-preview', 'choices': [{

0: 10
{'id': 'chatcmpl-9NF2pyLEwNlEVJ6XfI3EovcSRMy8h', 'object': 'chat.completion', 'created': 1715326403, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\nObject1: [cube, red, medium, matte],\nObject2: [cube, purple, medium, matte],\nObject3: [sphere, yellow, small, matte],\nObject4: [sphere, black, medium, metallic],\nObject5: [cylinder, teal, large, metallic],\nObject6: [cube, purple, large, matte],\nObject7: [cube, blue, large, matte],\nObject8: [cube, teal, medium, metallic],\nObject9: [sphere, purple, medium, matte],\nObject10: [cylinder, purple, large, matte],\nObject11: [cube, brown, medium, matte],\nObject12: [sphere, yellow, small, matte],\nObject13: [cube, purple, large, metallic],\nObject14: [sphere, gold, medium, metallic],\nObject15: [cylinder, brown, small, matte],\nObject16: [cube, red, large, matte],\nObject17: [cube, purple, small, matte],\nObject18: [cylinder, purple, large, matte],\nObject19: [cube, gray,

0: 18
{'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_9238278fa1e29fb954e3c4c9c9a7be82 in your email.)', 'type': 'server_error', 'param': None, 'code': None}}
0: 19
{'id': 'chatcmpl-9NF8CJkg1wAS1rrqPHFk6NJqzJG9z', 'object': 'chat.completion', 'created': 1715326736, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{Object1: [sphere, green, small, metallic], Object2: [cylinder, yellow, small, matte], Object3: [cylinder, purple, small, metallic], Object4: [cube, teal, small, matte], Object5: [cube, gray, small, metallic], Object6: [cylinder, yellow, medium, matte], Object7: [cylinder, yellow, small, matte], Object8: [sphere, teal, small, metallic], Object9: [sphere, brown, small, metallic], Object10: [cube, yellow, medium, matte], Object11: 

0: 28
{'id': 'chatcmpl-9NFDYtBTUtcwM9A3LzkTqo0jy65YL', 'object': 'chat.completion', 'created': 1715327068, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\nObject1: [small, yellow, sphere, matte],\nObject2: [small, brown, cube, matte],\nObject3: [medium, blue, cube, metallic],\nObject4: [medium, green, sphere, matte],\nObject5: [small, purple, sphere, metallic],\nObject6: [medium, purple, cube, matte],\nObject7: [medium, green, cylinder, matte],\nObject8: [small, brown, cube, metallic],\nObject9: [small, teal, sphere, metallic],\nObject10: [medium, purple, cube, metallic],\nObject11: [medium, green, cylinder, metallic],\nObject12: [small, teal, cube, matte],\nObject13: [medium, gray, cube, metallic],\nObject14: [medium, yellow, cylinder, metallic],\nObject15: [medium, yellow, sphere, matte],\nObject16: [large, yellow, sphere, matte],\nObject17: [medium, green, sphere, matte],\nObject18: [medium, brown, cylinder, matte],\nObj

0: 36
{'id': 'chatcmpl-9NFIZQ2KWQsxITIfpbmzrtCddFAen', 'object': 'chat.completion', 'created': 1715327379, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\nObject1: [cube, brown, medium, metallic],\nObject2: [cube, green, small, matte],\nObject3: [cylinder, yellow, medium, metallic],\nObject4: [cylinder, yellow, large, matte],\nObject5: [sphere, blue, medium, metallic],\n\nObject6: [cube, brown, small, matte],\nObject7: [cube, red, medium, matte],\nObject8: [sphere, yellow, small, matte],\nObject9: [cylinder, green, large, matte],\nObject10: [sphere, green, medium, matte],\n\nObject11: [sphere, green, large, metallic],\nObject12: [sphere, red, medium, matte],\nObject13: [cube, gray, medium, metallic],\nObject14: [sphere, blue, small, matte],\nObject15: [cylinder, teal, medium, metallic],\n\nObject16: [sphere, green, small, matte],\nObject17: [sphere, red, small, metallic],\nObject18: [cube, black, medium, matte],\nObject19: 

...

15: 194
{'id': 'chatcmpl-9NQbDIRFkgR5FHSc6sJGEhWiwZTQN', 'object': 'chat.completion', 'created': 1715370819, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\nObject1: [cube, blue, medium, metallic],\nObject2: [cube, green, small, matte],\nObject3: [cube, brown, small, matte],\nObject4: [cube, blue, large, matte],\nObject5: [cube, teal, large, metallic],\nObject6: [cube, green, small, matte],\nObject7: [cube, teal, medium, metallic],\nObject8: [cone, red, medium, matte],\nObject9: [sphere, yellow, medium, metallic],\nObject10: [cube, teal, large, metallic]\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 646, 'completion_tokens': 122, 'total_tokens': 768}, 'system_fingerprint': None}
15: 195
{'id': 'chatcmpl-9NQbmBBjU90ed6cCtZDkmXD6o7fOJ', 'object': 'chat.completion', 'created': 1715370854, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '

15: 204
{'id': 'chatcmpl-9NQh4z5cvbpnJ6TnW8HM18TQYOFoq', 'object': 'chat.completion', 'created': 1715371182, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Based on the provided image and following the format you've requested, here is the list of objects and their sub-properties:\n\n{\nObject1: [cube, blue, small, matte],\nObject2: [sphere, yellow, small, matte],\nObject3: [sphere, teal, medium, metallic],\nObject4: [sphere, blue, medium, metallic],\nObject5: [sphere, red, large, metallic]\n}"}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 646, 'completion_tokens': 86, 'total_tokens': 732}, 'system_fingerprint': None}
15: 205
{'id': 'chatcmpl-9NQheqn40KVJWel8ygYQz38YG2RYt', 'object': 'chat.completion', 'created': 1715371218, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{Object1: [cube, purple, small, matte], Object2: [sphere, blue, mediu

15: 215
{'id': 'chatcmpl-9NQnNoBkEghHZNptNQXVwlzhD2rOP', 'object': 'chat.completion', 'created': 1715371573, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{Object1: [sphere, purple, large, metallic], Object2: [cube, red, large, matte], Object3: [cylinder, teal, medium, matte], Object4: [cube, green, small, metallic], Object5: [cylinder, teal, large, matte]\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 646, 'completion_tokens': 64, 'total_tokens': 710}, 'system_fingerprint': None}
15: 216
{'id': 'chatcmpl-9NQnvtS9VGeEAXAq5dLRRsxQQ9gpH', 'object': 'chat.completion', 'created': 1715371607, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\nObject1: [cube, brown, medium, matte],\nObject2: [sphere, teal, small, metallic],\nObject3: [sphere, red, small, metallic],\nObject4: [sphere, black, large, metallic],\nObject5: [sphere, gold, medium, 

15: 227
{'id': 'chatcmpl-9NQu6Ki5SEWmTGFNPW8ImC5uMKGmh', 'object': 'chat.completion', 'created': 1715371990, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{Object1: [cube, gray, medium, matte], Object2: [cube, teal, medium, metallic], Object3: [sphere, teal, medium, matte], Object4: [cube, green, small, matte], Object5: [sphere, brown, small, matte],\nObject6: [cube, brown, small, matte], Object7: [sphere, blue, small, matte], Object8: [cube, blue, medium, matte], Object9: [sphere, blue, medium, matte], Object10: [cube, brown, medium, matte], \nObject11: [sphere, purple, small, matte], Object12: [sphere, teal, medium, metallic], Object13: [sphere, red, small, matte]\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 646, 'completion_tokens': 159, 'total_tokens': 805}, 'system_fingerprint': None}
15: 228
{'id': 'chatcmpl-9NQufJ1k6x2IiChfSozHxbu52ex5b', 'object': 'chat.completion', 'created': 1715372

15: 238
{'id': 'chatcmpl-9NR0MWRSjVZuxkGFjaDyWIigGqnKp', 'object': 'chat.completion', 'created': 1715372378, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{Object1: [cube, blue, medium, matte], Object2: [cube, gray, medium, matte], Object3: [cylinder, green, large, metallic], Object4: [sphere, green, small, matte], Object5: [cylinder, teal, medium, metallic]}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 646, 'completion_tokens': 63, 'total_tokens': 709}, 'system_fingerprint': None}


In [177]:
with open(f'{args.save_path}/responses_description.pkl', 'wb') as f:
    pickle.dump(responses_description_new, f)

In [130]:
# responses_description_old = responses_description_new.copy()
# responses_description_new = {**responses_description_old, **d}
# responses_description = responses_description_new.copy()

In [176]:
responses_description_new = {0: responses_description[0], 1: responses_description[1], 2: d[2], 3: responses_description[3]}
# d[2].keys()

In [160]:
responses_description_new[2][0][0]

{'id': 'chatcmpl-9N5Q0m0CGcNHUw6bpSrpnaz3pr1LC',
 'object': 'chat.completion',
 'created': 1715289400,
 'model': 'gpt-4-1106-vision-preview',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '{Object1: [blue, large, cylinder, matte], Object2: [teal, large, cube, shiny], Object3: [yellow, large, cube, matte], Object4: [red, large, cube, shiny], Object5: [teal, large, cylinder, matte], Object6: [gray, large, cylinder, matte], Object7: [brown, large, cylinder, matte], Object8: [yellow, large, cube, shiny], Object9: [brown, large, cube, shiny], Object10: [purple, large, cylinder, matte]}'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 646,
  'completion_tokens': 123,
  'total_tokens': 769},
 'system_fingerprint': None}

In [175]:
for cluster_id in range(12, 16):
    image_path = f"{args.base_path}/clustered_exemplars/block{cluster_id}*.png"
    exemplar_paths = glob.glob(image_path)
    exemplar_paths = natsorted(exemplar_paths)
    n_clusters = len(exemplar_paths)
    for k in range(0, n_clusters):
        d[2][cluster_id][k] = {}

In [141]:
responses_description[2] = d

## 3. Analyse if concept should be deleted

In [178]:
# load property list responses
with open(f'{args.save_path}/responses_knowledge_properties.pkl', 'rb') as f:
    knowledge_responses = pickle.load(f)
# load image descriptions based on these property lists
with open(f'{args.save_path}/responses_description.pkl', 'rb') as f:
    responses_description = pickle.load(f)

In [180]:
def extract_obj_list_from_response(response):
    try:
        # get object dict as str from response
        try:
            obj_dict_str = response.json()['choices'][0]['message']['content'].split('{')[-1].split('}')[0]
        except:
            obj_dict_str = response['choices'][0]['message']['content'].split('{')[-1].split('}')[0]
        # extract only property description lists
        obj_list_str = re.split(r'Object\d+: ', obj_dict_str)[1:]
        obj_list_str = [re.split(r'], ', s)[0]+']' for s in obj_list_str] 
    #     obj_list_str = [s.split('[')[1].split(']')[0].split(', ') for s in obj_list_str]
    # in case gpt didn't want to answer
    except:
        obj_list_str = ['']
    return obj_list_str


def extract_property_lists(response):
    # given response get only the property list relevant string
    property_str = extract_properties_from_response(response)
    # now clean it such that we have two arrays of strings, with each string representing a sub-property
    property_str = property_str.split('{')[-1].split(',}')[0]
    property_str = re.split(r': ', property_str)[1:]
    property_str = [s.split('[')[-1].split(']')[0] for s in property_str]
    property_str = [s.split(', ') for s in property_str]    
    return property_str


def get_bool_prop_list_for_response(response, knowledge_response, verbose=0):
    
    # get the object list contained within it
    obj_list_str =  extract_obj_list_from_response(response)
    if verbose:
        print(obj_list_str)

    # get the property list that the description prompt was based on
    property_list_str = extract_property_lists(knowledge_response)

    # iterate over each property and subproperty and check if it occurs in the object description of each object
    # properties_obj_bool represents a dictionary where each property id key contains a boolean array as value
    properties_obj_bool = {}
    for property_id in range(len(property_list_str)):
        # booolean array, for each object we store if it depicts a property
        property_obj_bool = np.zeros((len(obj_list_str), len(property_list_str[property_id])))

        for sub_property_id in range(len(property_list_str[property_id])):

            for obj_id in range(len(obj_list_str)):

                property_obj_bool[obj_id, sub_property_id] = \
                    property_list_str[property_id][sub_property_id] in obj_list_str[obj_id]

        properties_obj_bool[property_id] = property_obj_bool
    
    return properties_obj_bool


def check_for_deletion(prop_obj_bool_dict):
    delete_bool = []
    for prop_id in prop_obj_bool_dict.keys():
        # count the number of occurances of each subproperty over the objects
        occur_subprop = np.sum(prop_obj_bool_dict[prop_id], axis=0)
        # check if each object has one sub-property, i.e. the occurance of a subproperty equals the number of objs
        delete_bool.append(prop_obj_bool_dict[prop_id].shape[0] in occur_subprop)
    # if at least one subproperty is present across all objects return true
    if np.sum(delete_bool) > 0:
        return False
    else:
        return True
   

N_QUERIES = 4
# a little messed up in the order here, but for each block and each concept we now collect the deletion 
# booleans across samples
delete_block_dict = {}
for block_id in responses_description[0].keys():
    delete_concept_dict = {}
    for concept_id in responses_description[0][block_id].keys():
        delete_samples = []
#         for sample_id in responses_description.keys():
        for sample_id in range(N_QUERIES):
#             print(sample_id)
#             print(block_id)
#             print(concept_id)
            # get description resonse
            response = responses_description[sample_id][block_id][concept_id]
            # get a boolean representation of the object description list, where per object we check if a subproperty is in 
            # the corresponding obj list
            # prop_obj_bool_dict[0]: N_Obj x N_SubProps
            prop_obj_bool_dict = get_bool_prop_list_for_response(response, knowledge_responses[sample_id])
            # check if any subprop occurs across all objects
            delete_bool = check_for_deletion(prop_obj_bool_dict)

            delete_samples.append(delete_bool)

        delete_concept_dict[concept_id] = delete_samples

    delete_block_dict[block_id] = delete_concept_dict
    
delete_concepts_dict = {}
# now we perform majority vote over the collected deletion booleans
for block_id in delete_block_dict.keys():
    delete_concepts = []
    for concept_id in delete_block_dict[block_id].keys():
        vote_count = Counter(delete_block_dict[block_id][concept_id])
        top_two = vote_count.most_common(2)
        if len(top_two)>1 and top_two[0][1] == top_two[1][1]:
            # It is a tie, in this case we choose benefit of the doubt, i.e., don't delete
            pass
        # if majarity vote is true we delete the concept
        elif top_two[0][0]:
            delete_concepts.append(concept_id)
    delete_concepts_dict[block_id] = delete_concepts

delete_concepts_dict

{0: [1,
  2,
  3,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  61,
  62,
  63,
  64,
  65,
  66,
  68,
  69,
  70,
  71,
  72,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119],
 1: [0,
  1,
  6,
  11,
  13,
  14,
  16,
  17,
  20,
  21,
  23,
  24,
  25,
  26,
  27,
  30,
  31,
  32,
  34,
  35,
  36,
  37,
  39,
  40,
  41,
  45,
  46,
  48,
  49,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  60,
  61,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  72,
  73,
  74,


In [181]:
delete_block_dict

{0: {0: [True, False, False, False],
  1: [True, True, True, True],
  2: [True, True, True, True],
  3: [True, True, False, True],
  4: [False, True, False, True],
  5: [True, True, True, True],
  6: [False, True, True, True],
  7: [True, True, True, True],
  8: [True, True, True, True],
  9: [True, True, True, True],
  10: [True, True, True, True],
  11: [True, True, True, True],
  12: [True, True, True, True],
  13: [True, True, True, True],
  14: [True, True, True, True],
  15: [True, True, True, True],
  16: [True, True, True, True],
  17: [True, True, True, True],
  18: [True, True, True, True],
  19: [True, True, True, True],
  20: [True, True, True, True],
  21: [True, True, True, True],
  22: [True, True, True, True],
  23: [False, True, True, True],
  24: [True, True, True, True],
  25: [True, True, True, True],
  26: [True, True, True, True],
  27: [True, True, False, True],
  28: [True, True, True, True],
  29: [True, True, True, True],
  30: [True, True, True, True],
  31: 

In [182]:
with open(f'{args.save_path}/concepts_delete_dict.pkl', 'wb') as f:
    pickle.dump(delete_concepts_dict, f)

### 3.1 Now integrate this feedback into the retrieval corpus

In [152]:
# with open(f'{args.base_path}/block_concept_dicts.pkl', 'rb') as f:
#     ret_corpus = pickle.load(f)
# with open(f'{args.save_path}/concepts_delete_dict.pkl', 'rb') as f:
#     delete_concepts_dict = pickle.load(f)

In [153]:
# def set_id_from_id_list(id_list, delete_id, set_id=-1):
#     """
#     This function takes a list of ids, a delete_id which should be deleted and a set_id, i.e., the value which the
#     deleted ids will be set to instead.
#     """
#     # identify the ids of the concept-to-be-deleted
#     rel_ids = id_list == delete_id
#     # set these to -1
#     id_list[rel_ids] = set_id
#     return None


# def set_id_over_all_representations(block_corpus, delete_id, set_id=-1):
#     """
#     Iterates over all representations and resets the ids of the cluster id identified as 'delete_id'. 
#     'set_id' is the novel id which the cluster encodings are set to.
#     """
#     set_id_from_id_list(block_corpus['prototypes']['ids'], 
#                         delete_id=delete_id, 
#                         set_id=set_id)
#     set_id_from_id_list(block_corpus['exemplars']['ids'], 
#                         delete_id=delete_id, 
#                         set_id=set_id)
#     set_id_from_id_list(block_corpus['sivm_basis']['ids'], 
#                         delete_id=delete_id, 
#                         set_id=set_id)


# def remove_concept(block_corpus, delete_id):
#     """
#     Removes encodings and corresponding information entirely from the cluster identified via 'delete_id'
#     """
#     # i.e. 'prototypes', 'exemplars', 'sivm_basis'
#     representation_keys = list(block_corpus.keys())
#     representation_keys.remove('params_clustering')

#     for representation_key in representation_keys:
#         # e.g. 'exemplars', 'exemplar_ids'
#         data_keys = list(block_corpus[representation_key].keys())
#         # we handle the ids separately
#         data_keys.remove('ids')
#         # identify which encodings to keep and which not to keep
#         del_ids = np.where(block_corpus[representation_key]['ids'] == delete_id)[0]
#         keep_ids = np.where(block_corpus[representation_key]['ids'] != delete_id)[0]
#         # number of individual clusters altogether
#         n_clusters = len(np.unique(block_corpus[representation_key]['ids']))

#         for data_key in data_keys:
#             # handle the case where we have a list of arrays (e.g. 'exemplar_ids') vs a long list (e.g. 'exemplars')
#             if '_ids' in data_key:
#                 #len(block_corpus[representation_key][data_key]) == n_clusters:
#                 block_corpus[representation_key][data_key] = [
#                     ele for idx, ele in enumerate(block_corpus[representation_key][data_key]) if idx != delete_id
#                 ]
#             else:
#                 block_corpus[representation_key][data_key] = [
#                     ele for idx, ele in enumerate(block_corpus[representation_key][data_key]) if idx not in del_ids
#                 ]
#         # finally remove the ids themselves, i.e. keep only relevant ones
#         block_corpus[representation_key]['ids'] = block_corpus[representation_key]['ids'][keep_ids]

    
# # update ids in retrieval corpus based on feedback
# ret_corpus_delete = ret_corpus.copy()
# for block_id in delete_concepts_dict.keys():

#     # identify how many clusters exist per block
#     image_path = f"{args.base_path}/clustered_exemplars/block{block_id}*.png"
#     exemplar_paths = glob.glob(image_path)
#     exemplar_paths = natsorted(exemplar_paths)
#     n_clusters_block = len(exemplar_paths)
    
#     print(block_id)
#     print(delete_concepts_dict[block_id])
    
#     # we now have three cases how to handle deletion
#     # case 1: all clusters are to be deleted --> we set all cluster ids to 0
#     # case 2: all clusters, but 1 are to be deleted --> we merge all to-delete clsuters, 
#     #         i.e., set all to-delete cluster ids to one of this set
#     # case 3: at least two clusters should not be deleted --> we remove the cluster encodings completely of the
#     #         to-delete clusters
#     if delete_concepts_dict[block_id]:
#         # case 1
#         if len(delete_concepts_dict[block_id]) == n_clusters_block:
#             print(f'Block {block_id}: case 1')
#             # set cluster id in corpus to 0 for all representations
#             for delete_concept_id in delete_concepts_dict[block_id]:
#                 set_id_over_all_representations(
#                     ret_corpus_delete[block_id], 
#                     delete_id=delete_concept_id, 
#                     set_id=0
#                 )
#         # case 2
#         elif len(delete_concepts_dict[block_id]) == (n_clusters_block - 1): 
#             print(f'Block {block_id}: case 2')
#             # set all to-delete cluster ids to that of first one to delete, essentially merging these
#             set_id = delete_concepts_dict[block_id][0]
#             for delete_concept_id in delete_concepts_dict[block_id]:
#                 set_id_over_all_representations(
#                     ret_corpus_delete[block_id], 
#                     delete_id=delete_concept_id, 
#                     set_id=set_id
#                 )
#         # case 3
#         elif len(delete_concepts_dict[block_id]) <= (n_clusters_block - 2):
#             print(f'Block {block_id}: case 3')
#             for delete_concept_id in delete_concepts_dict[block_id]:
# #                 remove_concept(
# #                     ret_corpus_delete[block_id],
# #                     delete_id=delete_concept_id,
# #                 )
#                 set_id_over_all_representations(
#                     ret_corpus_delete[block_id], 
#                     delete_id=delete_concept_id, 
#                     set_id=-1
#                 )


0
[0, 1, 2, 4, 6, 8, 9, 11, 14, 15, 17, 18, 19, 21, 23, 24, 25, 27, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118, 119, 120, 122, 123, 124, 126, 127, 128, 129, 131, 132, 135, 136, 137, 138, 139, 141, 142, 143, 145, 146, 147, 148, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169, 171, 172, 173, 174, 175, 176, 178, 179, 180, 181, 182, 183, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 201, 202, 203, 204, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243]
Block 0: case 3
1
[1, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 33, 40]
Block 

In [154]:
# with open(f'{args.save_path}/block_concept_dicts_revise_delete.pkl', 'wb') as f:
#     pickle.dump(ret_corpus_delete, f)

## 4. Analyse if concepts should be merged

In [183]:
# load property list responses
with open(f'{args.save_path}/responses_knowledge_properties.pkl', 'rb') as f:
    knowledge_responses = pickle.load(f)
# load image descriptions based on these property lists
with open(f'{args.save_path}/responses_description.pkl', 'rb') as f:
    responses_description = pickle.load(f)
with open(f'{args.save_path}/concepts_delete_dict.pkl', 'rb') as f:
    delete_concepts_dict = pickle.load(f)

In [184]:
def get_common_properties_bool(prop_obj_bool_dict):
    common_properties = {}
    for prop_id in prop_obj_bool_dict.keys():
        # count the number of occurances of each subproperty over the objects
        occur_subprop = np.sum(prop_obj_bool_dict[prop_id], axis=0)
        # check if any sub-property occurs for all objs, returns bool array
        common_properties[prop_id] = occur_subprop == prop_obj_bool_dict[prop_id].shape[0]
    return common_properties


N_QUERIES = 4

# identify the common subproperties per concept
common_props = {}
for sample_id in range(N_QUERIES):
    common_props_block = {}
    for block_id in responses_description[0].keys():
        common_props_concept = {}
        for concept_id in responses_description[0][block_id].keys():
            if concept_id not in delete_concepts_dict[block_id]:
                # get description resonse
                response = responses_description[sample_id][block_id][concept_id]
                # get a boolean representation of the object description list, where per object we check if a subproperty is in 
                # the corresponding obj list
                # prop_obj_bool_dict[0]: N_Obj x N_SubProps
                prop_obj_bool_dict = get_bool_prop_list_for_response(response, knowledge_responses[sample_id])
                # # check if any subprop occurs across all objects
                # delete_bool = check_for_deletion(prop_obj_bool_dict)
                common_props_concept[concept_id] = get_common_properties_bool(prop_obj_bool_dict)
        common_props_block[block_id] = common_props_concept
    common_props[sample_id] = common_props_block

# now do pairwaise comparisons between remaining concepts
common_props_pairwise = {}
for sample_id in range(N_QUERIES):
# for sample_id in [0]:
    common_props_pairwise_block = {}
    for block_id in responses_description[0].keys():
#     for block_id in [15]:
        common_props_pairwise_concept1 = {}
        # get list of concept ids in this block
        concept_ids_list = list(common_props[0][block_id].keys())
        # and a copy of this
        concept_ids_2_list = list(common_props[0][block_id].keys())
        # we now perform a lower traingular pairwise comparison
        for concept_id1 in concept_ids_list:
            # remove the current concept_id1 from the second list
            concept_ids_2_list.remove(concept_id1)
            common_props_pairwise_concept2 = {}
            for concept_id2 in concept_ids_2_list:
                # gather the common props of the two concepts
                concept1 = common_props[sample_id][block_id][concept_id1]
                concept2 = common_props[sample_id][block_id][concept_id2]
#                 # check if they have a pairwise common subprop
#                 common_subprops = []
#                 for prop_id in concept1.keys():
#                     # test if any concept shares a common subproperty with the other concept
#                     common_subprops.append(np.sum((concept1[prop_id].astype(int)+concept2[prop_id].astype(int)) > 1))
#                 # test if any concept shares a common subproperty with the other concept
#                 common_props_pairwise_concept2[concept_id2] = int(np.sum(common_subprops) > 0)
                # if the two clusters share exactly the same distribution of subproperties then merge
                common_props_pairwise_concept2[concept_id2] = all(
                    ((concept2.get(k) == v).all() for k, v in concept1.items())
                )
            common_props_pairwise_concept1[concept_id1] = common_props_pairwise_concept2
        common_props_pairwise_block[block_id] = common_props_pairwise_concept1
    common_props_pairwise[sample_id] = common_props_pairwise_block



In [185]:
N_QUERIES = 4

merge_block_dict_samples = {}
for block_id in common_props_pairwise[0].keys():
    merge_concept_dict_samples = {}
    for concept_id in common_props_pairwise[0][block_id].keys():
        merge_concept2_dict_samples = {}
        for concept_id2 in common_props_pairwise[0][block_id][concept_id].keys():
            tmp = []
            for sample_id in range(N_QUERIES):
                tmp.append(common_props_pairwise[sample_id][block_id][concept_id][concept_id2])
            merge_concept2_dict_samples[concept_id2] = tmp
        merge_concept_dict_samples[concept_id] = merge_concept2_dict_samples
    merge_block_dict_samples[block_id] = merge_concept_dict_samples
    
merge_block_dict = {}
for block_id in merge_block_dict_samples.keys():
    merge_concept_dict = {}
    for concept_id in merge_block_dict_samples[block_id].keys():
        merge_concept2_dict = {}
        for concept_id2 in merge_block_dict_samples[block_id][concept_id].keys():
            # check if at least 2 queries produced a merge request
            merge_concept2_dict[concept_id2] = 0
            if sum(merge_block_dict_samples[block_id][concept_id][concept_id2]) >= 2:
                merge_concept2_dict[concept_id2] = 1
        merge_concept_dict[concept_id] = merge_concept2_dict
    merge_block_dict[block_id] = merge_concept_dict

In [186]:
merge_block_dict

{0: {0: {4: 0, 60: 0, 67: 0, 73: 1, 95: 1},
  4: {60: 1, 67: 0, 73: 0, 95: 0},
  60: {67: 0, 73: 1, 95: 0},
  67: {73: 0, 95: 0},
  73: {95: 0},
  95: {}},
 1: {2: {3: 0,
   4: 0,
   5: 0,
   7: 0,
   8: 0,
   9: 0,
   10: 0,
   12: 0,
   15: 0,
   18: 0,
   19: 0,
   22: 0,
   28: 0,
   29: 0,
   33: 0,
   38: 0,
   42: 0,
   43: 0,
   44: 0,
   47: 0,
   50: 0,
   59: 0,
   62: 0,
   71: 0,
   83: 0,
   84: 0,
   89: 0,
   91: 0,
   96: 0,
   99: 0,
   102: 0,
   118: 0,
   126: 0,
   134: 0,
   146: 0,
   149: 0,
   150: 0,
   151: 0,
   159: 0,
   166: 0,
   168: 0,
   170: 0,
   172: 0,
   174: 0,
   178: 0,
   186: 0,
   197: 0,
   198: 0,
   199: 1,
   200: 1,
   207: 0,
   216: 0,
   224: 0,
   239: 0,
   312: 0,
   327: 0,
   374: 0,
   389: 0,
   409: 0,
   420: 0,
   426: 0,
   431: 0},
  3: {4: 0,
   5: 0,
   7: 1,
   8: 0,
   9: 0,
   10: 0,
   12: 0,
   15: 0,
   18: 0,
   19: 0,
   22: 0,
   28: 0,
   29: 0,
   33: 0,
   38: 0,
   42: 0,
   43: 0,
   44: 0,
   47: 0,
   

In [187]:
with open(f'{args.save_path}/concepts_merge_samples_dict.pkl', 'wb') as f:
    pickle.dump(merge_block_dict_samples, f)
with open(f'{args.save_path}/concepts_merge_dict.pkl', 'wb') as f:
    pickle.dump(merge_block_dict, f)

### 4.1 Now integrate this feedback into the retrieval corpus

In [160]:
# with open(f'{args.save_path}/block_concept_dicts_revise_delete.pkl', 'rb') as f:
#     ret_corpus_delete = pickle.load(f)
# with open(f'{args.save_path}/concepts_merge_samples_dict.pkl', 'rb') as f:
#     merge_block_dict_samples = pickle.load(f)
# with open(f'{args.save_path}/concepts_merge_dict.pkl', 'rb') as f:
#     merge_block_dict = pickle.load(f)

In [161]:
# def merge_id_with_id2_in_id_list(id_list, to_be_merged_id, merge_to_id):
#     """
#     This function takes a list of ids, the id that should be merged and one to merge to. 
#     """
#     # identify the ids of the concept-to-be-deleted
#     rel_ids = id_list == to_be_merged_id
#     # set these to -1
#     id_list[rel_ids] = merge_to_id
#     return None

# # update ids in retrieval corpus based on feedback
# # i.e. if the majority of gpt prompts has identified a potential merge
# # replace the compared-to-concept-id (merge_concept_id2) with the comparing-concept-id (merge_concept_id)
# ret_corpus_delete_merge = ret_corpus_delete.copy()
# for block_id in merge_block_dict.keys():
#     for merge_concept_id in merge_block_dict[block_id].keys():
#         for merge_concept_id2 in merge_block_dict[block_id][merge_concept_id].keys():
#             if merge_block_dict[block_id][merge_concept_id][merge_concept_id2]:
#                 merge_id_with_id2_in_id_list(ret_corpus_delete_merge[block_id]['prototypes']['ids'], 
#                                              to_be_merged_id=merge_concept_id2, 
#                                              merge_to_id=merge_concept_id)
#                 merge_id_with_id2_in_id_list(ret_corpus_delete_merge[block_id]['exemplars']['ids'], 
#                                              to_be_merged_id=merge_concept_id2, 
#                                              merge_to_id=merge_concept_id)
#                 merge_id_with_id2_in_id_list(ret_corpus_delete_merge[block_id]['sivm_basis']['ids'], 
#                                              to_be_merged_id=merge_concept_id2, 
#                                              merge_to_id=merge_concept_id)

In [162]:
# with open(f'{args.save_path}/block_concept_dicts_revise_delete_merge.pkl', 'wb') as f:
#     pickle.dump(ret_corpus_delete_merge, f)

## (5. Make final query if the concepts should be merged?)

In [ ]:
# # load property list responses
# with open(f'{args.save_path}/responses_knowledge_properties.pkl', 'rb') as f:
#     knowledge_responses = pickle.load(f)

In [ ]:
# query concepts explicitly here via the two images based on merge_block_dict_samples, 
# i.e. if there is any uncertainty ask gpt again
# merge_prompt = f"You are provided two images. Each image contains at most 25 subimages. Each subimage depicts one object. Each object represents a reflective geometric solid that is placed in a neutral gray background scene with a light source. Furthermore, each object has multiple properties, e.g., color. Each property can be subdivided into several sub-properties, e.g., green is a sub-property of the property color. The possible properties and sub-properties are the following: \n\n{response_str}\n\nFocusing only on these properties, please perform the following tasks.\n\nFirst, is there a common sub-property that is depicted by all objects over both images? Please provide your answer with a final Yes or No in the following format: Answer <your-descriptive-answer> Final Answer <your-final-answer>"

# Misc.

In [ ]:
#delete_prompt = """You are provided two images. An image contains subimages. Each subimage depicts one object. Each object represents a reflective geometric solid that is placed in a neutral gray background scene with a light source. Furthermore, each object has multiple properties, e.g., color. Each property can be subdivided into several sub-properties, e.g., green is a sub-property of the property color. 

# Please provide a list of obect properties and subproperties that are depicted in all images. Ignore the background and the object's luminance and reflectivity. Use the following answer template:

# {
# property: [sub-property, sub-property, ...]
# property: [sub-property, sub-property, ...]
# ...
# }

# Given this list is there a property that is shared by all objects in the second image which does not also occur for all objects of the first image? Please provide a conclusive, final answer that summarizes your answer with ONLY a 'Yes' or 'No'. It is ok to say no. Use this format: Property List: <your-list> Answer: <your-descriptive-answer> Final Answer: <your-final-answer>"""
# First, list the common sub-properties of the objects in the first image.\n\nSecond, list the common sub-properties of the objects in the second image. It is possible that no common sub-property exists.\n\nThird, given these lists, is there a commom sub-property between the objects of the second image which is not also a common sub-property between the objects of the first image? 

# delete_prompt = """You are GPT4, an AI model with the ability to view and describe images. You are provided two images. These two images are definitely available to you. An image contains subimages. Each subimage depicts one object. Each object represents a reflective geometric solid that is placed in a neutral gray background scene with a light source. Furthermore, each object has multiple properties, e.g., color. Each property can be subdivided into several sub-properties, e.g., green is a sub-property of the property color. 

# Please provide a list of obect properties and subproperties that are depicted in all images. Ignore the background and the object's luminance and reflectivity. Use the following answer template:

# {
# property: [sub-property, sub-property, ...]
# property: [sub-property, sub-property, ...]
# ...
# }

# Focusing only on these properties, is there a common sub-property of all of the objects in the second image? It is possible that no common sub-property exists.

# Please summarize your answer with a `Yes` or `No`. Please answer in the following in the format: Property List: <your-list> Answer: <your-descriptive-answer> Final Answer: <your-final-answer>."""
    
    
# old prompts:
# merge_prompt = "The following images contain objects that represent reflective geometric solids that are placed in a neutral gray background scene with a light source. Please describe the properties of all of the objects in each image individually. Now list all properties that all obejcts in all three images share. Finally, given this list and the previous description, is there a property that is shared by all objects in the second image and third image which does not also occur for all objects of the first image? Please provide a conclusive, final answer that summarizes your answer in a Yes or No. It is ok to say no. Use this format: Description: <your-descriptive-list> Answer: <your-descriptive-answer> Final Answer: <your-final-answer>"    
# merge_prompt = "The following images contain objects that represent reflective geometric solids that are placed in a neutral gray background scene with a light source. Please describe the properties of all of the objects in each image individually. Now list all properties that all obejcts in all three images share. Finally, given this list and the previous description, is there an object property that any object of the second image shares with any object of the third image which that object of the second image does not also share with any object of the first image? Please provide a conclusive, final answer that summarizes your answer in a Yes or No. It is ok to say no. Use this format: Description: <your-descriptive-list> Answer: <your-descriptive-answer> Final Answer: <your-final-answer>"    
## Given gpt4 knowledge
# merge_prompt = f"You are provided two different images. An image contains subimages. Each subimage depicts one object. Each object represents a reflective geometric solid that is placed in a neutral gray background scene with a light source. Furthermore, each object has multiple properties, e.g., color. Each property can be subdivided into several sub-properties, e.g., green is a sub-property of the property color. The possible properties and sub-properties are the following: \n\n{response_str}\n\nFocusing only on these properties, please perform the following tasks.\n\nFirst, list the common sub-properties of the objects in the first image.\n\nSecond, list the common sub-properties of the objects in the second image.\n\nGiven these lists, is there a commom sub-property between the objects of the first and second image? It is possible that no common sub-property exists. Please summarize your answer with a `Yes` or `No` in the format: Final Answer: <your-final-answer>."

## 1. Query VLM if there are concept clusters that can be deleted

In [83]:
# N_QUERY_SAMPLES = 7

# with open(f'{args.save_path}/responses_properties.pkl', 'wb') as f:
#     pickle.dump(knowledge_responses, f)

# responses_delete = {}
# for n_sample in range(N_QUERY_SAMPLES):

#     delete_prompt = f"You are provided an image. The image contains at most 25 subimages. Each subimage depicts one object. Each object represents a reflective geometric solid that is placed in a neutral gray background scene with a light source. Furthermore, each object has multiple properties, e.g., color. Each property can be subdivided into several sub-properties, e.g., green is a sub-property of the property color. The possible properties and sub-properties are the following: \n\n{response_str}\n\nFocusing only on these properties, please perform the following tasks.\n\nFirst, for every object in the image please list the sub-properties from the given lists that the object depicts. Only name the sub-properties that are given. Please use the following format:\n\n{{Object1: [sub-property, ...], Object2: [sub-property, ...], ...\n}}"

#     responses_block = {}
#     # Gather the responses per concept deletion query
#     # TODO: comment in !!!!!!
#     # for block_id in range(args.num_blocks):
#     for block_id in [1]:

#         image_path = f"{args.base_path}/clustered_exemplars/block{block_id}*.png"
#         exemplar_paths = glob.glob(image_path)
#         exemplar_paths = natsorted(exemplar_paths)
#         n_clusters = len(exemplar_paths)

#         if n_clusters == 1:
#             pass

#         responses_concept = {}
#         for concept_id, img_pth in enumerate(exemplar_paths):

#             print(f'{block_id}: {concept_id}')

#             # TODO: comment out !!!!!!
#             if concept_id > 0:
#                 break

#             # Getting the base64 string
#             base64_concept_image = encode_image(img_pth)

#             payload = {
#               "model": "gpt-4-vision-preview",
#               "messages": [
#                 {
#                   "role": "user",
#                   "content": [
#                     {
#                       "type": "text",
#                       "text": f"{delete_prompt}",
#                     },
#                     {
#                       "type": "image_url",
#                       "image_url": {
#                         "url": f"data:image/jpeg;base64,{base64_concept_image}"
#                       }
#                     },
#                   ],
#                 }
#               ],
#               "max_tokens": 400
#             }

#             response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
#             responses_concept[concept_id] = response
#             print(response.json())
#             response_str = extract_obj_dict_from_response(response)
#             print(response_str)

#             time.sleep(10)

#         responses_block[block_id] = responses_concept

#     responses_delete[n_sample] = responses_block

1: 0
{'id': 'chatcmpl-9FH51zebbVncdG6HxPvQfyJ9fG6NM', 'object': 'chat.completion', 'created': 1713427603, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\nshape: [cone, sphere, cube, cylinder],\ncolor: [purple, red, gold, teal, green, blue, grey],\n}\n'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 912, 'completion_tokens': 30, 'total_tokens': 942}, 'system_fingerprint': None}
{shape: [cone, sphere, cube, cylinder],color: [purple, red, gold, teal, green, blue, grey],}
{'id': 'chatcmpl-9FH55sBxMX9dZ6JulDknY2PDD2pcL', 'object': 'chat.completion', 'created': 1713427607, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "To assist with your request, let’s analyze the objects based on the properties you've mentioned: color and shape.\n\nFirst Image:\n- Common shape(s): As the shapes vary, there seems to be no single shape in common across all obj

## 2. Summarize VLM responses if there are concept clusters that can be deleted

In [10]:
# delete_concepts = {}
# # TODO: comment in !!!!!!
# for block_id in range(args.num_blocks):
# # for block_id in [2]:

#     delete_concepts_for_block = []

#     n_clusters = len(responses_delete[block_id].keys())

#     if n_clusters == 1:
#         delete_concepts_for_block.append(0)
#     else:
#         for concept_id in range(n_clusters):
#             majority_answer = find_majority(responses_delete[block_id][concept_id], verbose=0)
#             if majority_answer:
#                 pass
#             else:
#                 delete_concepts_for_block.append(concept_id)

#     delete_concepts[block_id] = delete_concepts_for_block

In [12]:
# delete_concepts

{0: [0, 1, 2, 3],
 1: [0, 1, 2, 3],
 2: [0],
 3: [0],
 4: [0, 1],
 5: [0, 1, 2],
 6: [0, 3, 4, 5, 6],
 7: [1, 4, 5, 6, 7, 8]}

In [11]:
# with open(f'{args.save_path}/revision_delete_concepts.pkl', 'wb') as f:
#     pickle.dump(delete_concepts, f)

# with open(f'{args.save_path}/revision_delete_concepts_responses.pkl', 'wb') as f:
#     pickle.dump(responses_delete, f)

## 3. Query VLM if there are concept clusters that can be merged

In [220]:
# # TODO: comment in !!!!!!
# delete_concepts = {2: [2]}
# N_QUERY_SAMPLES = 3
# responses_merge = {}
# # Gather the 5 responses per concept merge query
# # for block_id in range(args.num_blocks):
# for block_id in [2]:

#     image_path = f"{args.base_path}/clustered_exemplars/block{block_id}*.png"
#     exemplar_paths = glob.glob(image_path)
#     exemplar_paths = natsorted(exemplar_paths)
#     n_clusters = len(exemplar_paths)

#     if n_clusters == 1:
#         pass

#     responses_for_block = {}
#     has_been_merged = []
#     # iterate over all concepts in a block
#     for concept_id, img_pth in enumerate(exemplar_paths):

#         # TODO: comment in !!!!!!
#         if concept_id > 3:
#             break

#         responses_for_concept = {}

#         # if the concept had not previously been identified to be deleted
#         if concept_id not in delete_concepts[block_id] and concept_id not in has_been_merged:

#             # Getting the base64 string of first concept image
#             print(img_pth)
#             base64_concept_image = encode_image(img_pth)

#             # TODO: comment in !!!!!!
# #             for concept_id_2 in range(n_clusters):
#             for concept_id_2 in [0, 1, 2, 3]:

#                 print(concept_id)
#                 responses_for_concept_pair = []

#                 # if the pair concept id is not the current one and has not been merged previously
#                 if concept_id_2 != concept_id and concept_id_2 not in has_been_merged and concept_id_2 not in delete_concepts[block_id]:
#                     print(concept_id_2)
#                     # sample a response N_QUERY_SAMPLES times
#                     for sample_id in range(N_QUERY_SAMPLES):

#                         # Getting the base64 string of second concept image
#                         base64_concept_image_2 = encode_image(exemplar_paths[concept_id_2])

#                         payload = {
#                           "model": "gpt-4-vision-preview",
#                           "messages": [
#                             {
#                               "role": "user",
#                               "content": [
#                                 {
#                                   "type": "text",
#                                   "text": f"{merge_prompt}",
#                                 },
#                                 {
#                                   "type": "image_url",
#                                   "image_url": {
#                                     "url": f"data:image/jpeg;base64,{BASE64_RAND_IMG}"
#                                   }
#                                 },
#                                 {
#                                   "type": "image_url",
#                                   "image_url": {
#                                     "url": f"data:image/jpeg;base64,{base64_concept_image}"
#                                   }
#                                 },
#                                 {
#                                   "type": "image_url",
#                                   "image_url": {
#                                     "url": f"data:image/jpeg;base64,{base64_concept_image_2}"
#                                   }
#                                 },
#                               ],
#                             }
#                           ],
#                           "max_tokens": 600
#                         }

#                         response = requests.post(
#                             "https://api.openai.com/v1/chat/completions", 
#                             headers=headers, 
#                             json=payload
#                         )
#                         responses_for_concept_pair.append(response)
#                         time.sleep(10)

#                     # on the fly compute the majority vote over the sampled responses
#                     try:
#                         majority_answer = find_majority(responses_for_concept_pair)
#                     except:
#                         print(responses_for_concept_pair)
#                     # if majority vote says yes then merge the two concepts and put the second concept id in 
#                     # the list of concept ids that should not be queried anymore
#                     if majority_answer:
#                         has_been_merged.append(concept_id_2)
#                     responses_for_concept[concept_id_2] = responses_for_concept_pair

#         responses_for_block[concept_id] = responses_for_concept

#     responses_merge[block_id] = responses_for_block

../logs/clevr_easy_500_epochs/sysbind_orig_seed0/clustered_exemplars/block2_0.png
0
0
1
I'm sorry, I can't provide assistance with these images.
[<Response [200]>, <Response [200]>, <Response [200]>]
0
0
3
I'm sorry, but I cannot provide assistance with these images.
[<Response [200]>, <Response [200]>, <Response [200]>]


## 4. Summarize VLM responses if there are concept clusters that can be merged

In [212]:
# merge_concepts = {}
# # for block_id in range(args.num_blocks):
# for block_id in [2]:
    
#     # !!! the semantics is: {i: j} concept i gets merged into concept j !!!
#     merge_concepts_for_block = {}

#     n_clusters = len(responses_merge[block_id].keys())

#     if n_clusters == 1:
#         pass
#     else:
#         for concept_id in range(n_clusters):
#             for concept_key in responses_merge[block_id][concept_id].keys():
#                 majority_answer = find_majority(responses_merge[block_id][concept_id][concept_key], verbose=0)
#                 if majority_answer:
#                     merge_concepts_for_block[concept_key] = concept_id
#                 else:
#                     pass

#     merge_concepts[block_id] = merge_concepts_for_block

I'm sorry, but I cannot assist with this request.


Exception: This response isn't binary

In [ ]:
# with open(f'{args.save_path}/revision_merge_concepts.pkl', 'wb') as f:
#     pickle.dump(merge_concepts, f)

# with open(f'{args.save_path}/revision_merge_concepts_responses.pkl', 'wb') as f:
#     pickle.dump(responses_merge, f)